- Automated generate test data

- https://docs.ragas.io/en/stable/getstarted/rag_testset_generation

In [1]:
from dotenv import load_dotenv
from ragas.testset import TestsetGenerator
from langchain_community.document_loaders import DirectoryLoader
from ragas.llms.base import llm_factory
from ragas.embeddings import OpenAIEmbeddings
import openai
import os
import random
import re

In [2]:
load_dotenv("configs.env")

True

In [3]:
TEST_DATA_DIR_PATH = "test_data_files"
os.makedirs(TEST_DATA_DIR_PATH, exist_ok=True)

OUTPUT_TEST_DATA_DIR = "generated_test_data"
os.makedirs(OUTPUT_TEST_DATA_DIR, exist_ok=True)

In [4]:
generator_llm = llm_factory("gpt-4o-mini")
openai_client = openai.AsyncOpenAI()
generator_embeddings = OpenAIEmbeddings(
    client=openai_client, model="text-embedding-3-small"
)

In [5]:
generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)

In [6]:
doc_files = DirectoryLoader(TEST_DATA_DIR_PATH).load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


In [7]:
len(doc_files)

2

In [8]:
dataset = generator.generate_with_langchain_docs(doc_files, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/21 [00:00<?, ?it/s]

Applying EmbeddingExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

...

Applying OverlapScoreBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Skipping multi_hop_abstract_query_synthesizer due to unexpected error: No relationships match the provided condition. Cannot form clusters.


Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

In [9]:
df = dataset.to_pandas()

In [10]:
df.shape

(10, 4)

In [11]:
df.head()

,user_input,reference_contexts,reference,synthesizer_name
0,What impression did Jonathan Harker have of Bu...,[JONATHAN HARKER’S JOURNAL (_Kept in shorthand...,Jonathan Harker described Buda-Pesth as a wond...,single_hop_specific_query_synthesizer
1,What is the Golden Krone Hotel like?,[saw little towns or castles on the top of ste...,The Golden Krone Hotel is described as thoroug...,single_hop_specific_query_synthesizer
2,What significance does St. George’s Day hold i...,[4 May._--I found that my landlord had got a l...,St. George’s Day is significant as it is descr...,single_hop_specific_query_synthesizer
3,"In the context of the narrative, what signific...",[5 May. The Castle._--The grey of the morning ...,'Ordog' translates to 'Satan' in the local lan...,single_hop_specific_query_synthesizer
4,Why the Hospadars no fix things for fear of th...,"[the Hospadars would not repair them, lest the...","The Hospadars would not repair them, lest the ...",single_hop_specific_query_synthesizer


In [12]:
os.makedirs(OUTPUT_TEST_DATA_DIR, exist_ok=True)

In [13]:
df.to_csv(os.path.join(OUTPUT_TEST_DATA_DIR, "test_data.csv"), index=False)